In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import Input

avg_rmse_ws_3, avg_rmse_ws_4, avg_rmse_ws_5 = [], [], []
avg_mape_ws_3, avg_mape_ws_4, avg_mape_ws_5 = [], [], []

def average(df, window_size, rmse, mape):
    avg_rmse = np.mean(rmse)
    avg_mape = np.mean(mape)
    print(f"order: {df['order'].iloc[0]} / window size: {window_size} ---- avg_rmse: {avg_rmse}, avg_mape: {avg_mape}")

    if window_size == 3:
        avg_rmse_ws_3.append(avg_rmse)
        avg_mape_ws_3.append(avg_mape)
    elif window_size == 4:
        avg_rmse_ws_4.append(avg_rmse)
        avg_mape_ws_4.append(avg_mape)
    elif window_size == 5:
        avg_rmse_ws_5.append(avg_rmse)
        avg_mape_ws_5.append(avg_mape)

    return avg_rmse_ws_3, avg_rmse_ws_4, avg_rmse_ws_5, avg_mape_ws_3, avg_mape_ws_4, avg_mape_ws_5

def train_model(x, y, next_x):
    x = np.array(x).reshape((x.shape[0], x.shape[1], 1))
    y = np.array(y).flatten()

    # model = Sequential([
    #     Input(shape=(x.shape[1], 1)),
    #     LSTM(50, activation='relu', return_sequences=True),
    #     LSTM(50, activation='relu', return_sequences=False),
    #     Dense(25, activation='relu'),
    #     Dense(1)
    # ])
    model = Sequential()
    model.add(LSTM(10, activation='tanh', input_shape=(x.shape[1], 1)))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse')

    model.fit(x, y, epochs=50, batch_size=8, verbose=0)

    next_x = np.array(next_x, dtype=np.float32).reshape((1, next_x.shape[0], 1))
    y_pred = model.predict(next_x)

    return y_pred[0][0]

def pred_mig(df, features, window_size):
    result = []
    rmse_array = []
    mape_array = []

    for count in range(len(df) - window_size):
        get_x = df.loc[2006+count : 2006+window_size+count-1, list(features)]
        get_y = df.loc[2006+count : 2006+window_size+count-1, "淨遷移率"]
        get_next_x = df.loc[2006+window_size+count, list(features)]
        get_next_y = df.loc[2006+window_size+count, "淨遷移率"]

        pred_next_y = train_model(get_x, get_y, get_next_x)

        rmse_score = np.sqrt(mean_squared_error([get_next_y], [pred_next_y]))
        mape_score = (abs((get_next_y - pred_next_y) / get_next_y)).mean()

        rmse_array.append(rmse_score)
        mape_array.append(mape_score)

        result.append([window_size, 2006+window_size+count, get_next_y, pred_next_y, rmse_score, mape_score])

    average(df, window_size, rmse_array, mape_array)
    return result

def forecast(df, features):
    window_sizes = [3, 4, 5]
    all_results = []

    for window_size in window_sizes:
        all_results.extend(pred_mig(df, features, window_size))

    print("\n")
    result_df = pd.DataFrame(all_results, columns=["window_size", "year", "actual_y", "predicted_y", "rmse", "mape"])
    return result_df

def main():
    excel_file = pd.ExcelFile("prefecture_analysis.xlsx")

    file_pca = "results_pca_lstm.xlsx"
    file_xgb = "results_xgb_lstm.xlsx"

    with pd.ExcelWriter(file_pca, engine="xlsxwriter") as writer_pca, \
         pd.ExcelWriter(file_xgb, engine="xlsxwriter") as writer_xgb:
        
        for sheet_name in excel_file.sheet_names:
            df = pd.read_excel("prefecture_analysis.xlsx", sheet_name=sheet_name)
            df = df.dropna()  # 移除包含 NaN 的行
            df.set_index("year", inplace=True)

            # pca_features = {"降雨量", "空屋率", "一級產業生產額", "降雨日數", "失業率", "有效求人倍數"}
            xgb_features = {"第1次産業就業者比例", "財政力指數", "新住宅動工數", "千人社會體育設施數", "縣內總生產", "大學生數"}

            # result_pca = forecast(df, pca_features)
            result_xgb = forecast(df, xgb_features)
            
            # result_pca.to_excel(writer_pca, sheet_name=sheet_name, index=False)
            result_xgb.to_excel(writer_xgb, sheet_name=sheet_name, index=False)

            print(f"{sheet_name} 預測結果已存入 {file_pca} 和 {file_xgb}")
        
        print("\n=== Final Results ===")
        print(f"avg_rmse_ws_3: {np.mean(avg_rmse_ws_3):.4f}")
        print(f"avg_rmse_ws_4: {np.mean(avg_rmse_ws_4):.4f}")
        print(f"avg_rmse_ws_5: {np.mean(avg_rmse_ws_5):.4f}")
        print(f"avg_mape_ws_3: {np.mean(avg_mape_ws_3):.4f}")
        print(f"avg_mape_ws_4: {np.mean(avg_mape_ws_4):.4f}")
        print(f"avg_mape_ws_5: {np.mean(avg_mape_ws_5):.4f}")

    print("所有 sheets 預測結果已儲存完成！")

if __name__ == '__main__':
    main()


1/1 [==============================] - 0s 221ms/step
order: 1 / window size: 3 ---- avg_rmse: 1.1379761989630637, avg_mape: 0.8838521710020574
1/1 [==============================] - 0s 222ms/step
order: 1 / window size: 4 ---- avg_rmse: 0.7029111977653325, avg_mape: 0.5298923175685629
1/1 [==============================] - 0s 214ms/step
order: 1 / window size: 5 ---- avg_rmse: 0.8956765099967973, avg_mape: 0.647643213290847


Order_1 預測結果已存入 results_pca_lstm.xlsx 和 results_xgb_lstm.xlsx
1/1 [==============================] - 0s 221ms/step
order: 2 / window size: 3 ---- avg_rmse: 3.1356125214289263, avg_mape: 0.791830619002336
1/1 [==============================] - 0s 219ms/step
order: 2 / window size: 4 ---- avg_rmse: 3.2099992222724754, avg_mape: 0.81301206583851
1/1 [==============================] - 0s 219ms/step
order: 2 / window size: 5 ---- avg_rmse: 3.4454166760874996, avg_mape: 0.8315008264000007


Order_2 預測結果已存入 results_pca_lstm.xlsx 和 results_xgb_lstm.xlsx
1/1 [=============

: 